## Criando o primeiros imports do simulador

In [ ]:
from quantumnet.components import Network, Controller, Host
from quantumnet.objects import Logger, Qubit

from random import randint, choice
from copy import copy

## Criando a rede

In [ ]:
def initNetwork(rows: int, 
                columns: int, 
                log: bool=True) -> tuple[Network, Controller]:
    """
    Will initiate the network and will define the controller

    Args:
        rows: Number of rows of host in the network
        columns: Number of columns of host in the network
        log: If True will activate logs od simulation  

    Returns:
        Will return the Network and the Controller
    """
    # Defining the network
    network = Network()

    # Defining the Controller
    controller = Controller(network)

    # Defining the topology
    network.set_ready_topology('Grade', rows, columns)

    # Draw Nodes
    network.draw()

    # Log of simulator
    if log:
        Logger.activate(Logger)
        
    return network, controller


## Escolhendo os nós do Black Hole

In [ ]:
def selectBlackHoles(network: Network, 
                    num_black_holes: int) -> list:
    """
    Randomly choose Black Holes

    Args:
        num_black_holes: Number of Black Holes

    Returns:
        Black Holes: List with all Black Holes
    """
    all_hosts = copy(network.get_all_sorted_hosts())
    black_hole_list = []
    if num_black_holes > 0 and num_black_holes < len(all_hosts):
        for host in range(0, num_black_holes):
            black_hole = choice(all_hosts)
            black_hole_list.append(black_hole)
            all_hosts.pop(black_hole.host_id)

    print(f"Lista de Black Holes: [ ", end='')
    for host in black_hole_list:
        print(f"{host.host_id} ", end='')
    print(']')

    return black_hole_list

## Escolhendo os nós de Alice e de Bob

In [ ]:
def selectAliceBob(network: Network, 
                    black_hole_list: list) -> tuple[Host, Host]:
    """
    Will define the nodes: Alice, Bob

    Args:
        network: Network to wich the nodes belong
        black_hole_list: List with all Black Holes

    Returns:
        Hosts: Will return respectively: Alice host, Bob host
    """
    valid = False
    while not valid:
        alice_id = randint(0, len(network.hosts)-1)
        alice = network.get_host(alice_id)
        if alice not in black_hole_list:
            valid = True
    print(f"O id de Alice é: {alice}")

    valid = False
    while not valid:
        bob_id = randint(0, len(network.hosts)-1)
        bob = network.get_host(bob_id)
        if bob_id != alice_id and bob not in black_hole_list:
            valid = True
    print(f"O id de Bob é: {bob}")


    return alice, bob


## Passando as rotas para o controlador

In [ ]:
def getRoute(network: Network, 
            controller: Controller, 
            alice: Host, 
            bob: Host) -> list:
    """
    Will define the route of Alice to Bob

    Args:
        network: Network to wich the nodes belong
        controller: Controller of Network
        alice: Sender host
        bob: Receiver host

    Returns:
        Route: List with the rotes of Alice to Bob
    """
    # Geting the all routing table
    controller.register_routing_tables()

    # Defining the route of Alice to Bob
    route = network.get_host(alice.host_id).routing_table[bob.host_id]
    print(f"Alice: {alice} deseja se comunicar com Bob: {bob} pela rota {route}")
    
    return route

## Modificando manualmente a probabilidade do Black_Hole

In [ ]:
def setNetworkSwappProb(network: Network, 
                        network_prob: float,  
                        malicious_hosts_list: Host, 
                        malicious_hosts_prob: float) -> None:
    """
    Will set network's probability of success of entanglement swapping

    Args:
        network: Network to wich the nodes belong
        network_prob: Network's entanglement swapping probability
        malicious_hosts: List with host which will attack the network
        malicious_hosts_prob: Malicious host probability
    """
    for host_id in range(0, len(network.hosts)):
        temp_host = network.get_host(host_id=host_id)

        if temp_host in malicious_hosts_list:
            temp_host.setEntanglementSwappingProb(malicious_hosts_prob)
        else:
            temp_host.setEntanglementSwappingProb(network_prob)


## Criação manual de entanglement entre os nós

In [ ]:
def addQubits(host_A: Host, 
                host_B: Host, 
                counter: int) -> int:
    """
    Will add qubits to both hosts

    Args:
        host_A: Host that wants to add the qubit
        host_B: Host that wants to add the qubit
        counter: Counter to index qubits

    Returns:
        Counter: Return updated counter
    """
    temp_qubit_counter = counter
    
    qubit = Qubit(temp_qubit_counter)
    host_A.add_qubit(qubit)

    qubit = Qubit(temp_qubit_counter+1)
    host_B.add_qubit(qubit)

    temp_qubit_counter += 2

    return temp_qubit_counter
    
# Creating entanglement between neighbors hosts
def createEntanglements(route: list, 
                        network: Network,
                        number_of_entanglements: int) -> None:
    """
    Create entangleds pairs between the hosts of route

    Args:
        route: list with Host_A and Host_B
        network: Network to wich the nodes belong
        number_of_entanglements: Number of desired pairs
    """
    # Used qubits
    qubit_counter = 0

    # Loop to create new entanglements
    for entanglement in range(0, number_of_entanglements):
        
        # Save the qubit index to create new qubits
        temp_qubit_counter = qubit_counter

        # Create new entanglement to every host in the route
        host_A = network.get_host(route[0])
        host_B = network.get_host(route[1])

        # Will trying until entangled be successfully created
        entangled = False
        while not entangled:

            # If dont't have qubit on memory will add
            if host_A.memory == [] or host_B.memory == []:
                temp_qubit_counter = addQubits(host_A=host_A, host_B=host_B, counter=temp_qubit_counter)
            print(f"Tentativa de entanglement entre {host_A} e {host_B}")

            # Trying do entanglement between host_A and host_B
            entangled = network.physical.entanglement_creation_heralding_protocol(host_A, host_B)

            if not entangled:
                temp_qubit_counter = addQubits(host_A=host_A, host_B=host_B, counter=temp_qubit_counter)
                
        # Update counter to qubit index
        qubit_counter += temp_qubit_counter

    print(f"Foram criados {qubit_counter} qubits a mais para a realização dos {entanglement+1} entanglements")

## Criando um reabastecimento da rede

In [ ]:
def replenishNetwork(network: Network, 
                    edges: list, 
                    number_of_entanglements) -> None:
    print("Repondo os recursos da rede")
    for edge in edges:
        createEntanglements(route=edge, network=network, number_of_entanglements=number_of_entanglements)

## Criando as requisições

In [ ]:
def createRequest(network: Network, alice: Host, bob: Host, attempts: int, route: list) -> tuple[int, int]:
    """
    Args:
        network: Network to wich the nodes belong
        alice: Sender host
        bob: Receiver host
        route: Route of Alice to Bob

    Returns:
        Will return entanglement result and counter of attempts
    """
    counter = 0
    for attempt in range(0, attempts):
        entangled = network.networklayer.entanglement_swapping(alice.host_id, bob.host_id, route=route)
        if entangled != 0:
            break
        counter += 1

    if entangled == -1:
        print("Não é possível realizar o entanglement swapping")
    elif entangled == 0:
        print(f"O entanglement falhou com o total de {attempts}")
    else:
        print(f"O entanglement foi um sucesso depois de {counter} tentativas")

    network.get_host(alice.host_id).announce_to_controller_app_has_finished()

    return entangled, counter

## Criando a simulação

In [ ]:
def simulation(
        log: bool, 
        rows: int, 
        columns: int, 
        entanglements_replanished: int = 0, 
        requests: int = 100,
        attempts_per_request: int = 2,
        network_prob: float = None, 
        num_black_holes: int = 1, 
        black_hole_prob: int = None
        ) -> dict:
        """Run the simulation with the desired parameters

            Args:
                runTimes: Number of times of simulation will run
                log: Will show the simulator logs
                rows: Number of rows of host in the network
                columns: Number of columns of host in the network 
                entanglements_replanished: Number of entangled pair will be create to replanish network
                requests: Number of requests in simulation
                attempts_per_request: Number of attempts on a request
                network_prob: Network's entanglement swapping probability
                num_black_holes: Number of Black Holes in the network
                black_hole_prob: Malicious host probability

            Returns:
                Dict: Return every information of run on simulation"""

        # Create network and controller
        network, controller = initNetwork(rows=rows, columns=columns, log=log)

        # Set real edges
        real_edges = network.edges

        # Select Black Hole list
        black_hole_list = selectBlackHoles(network=network, num_black_holes=num_black_holes)

        # Select network Prob
        setNetworkSwappProb(network=network, network_prob=network_prob, 
                            malicious_hosts_list=black_hole_list, malicious_hosts_prob=black_hole_prob)

        # Dict with requests data
        data = {}

        # Add hash to requests
        data["Requests"] = {}

        # Add Black Hole list
        data["Black Holes"] = [host.host_id for host in black_hole_list]

        # Run requests
        for request in range(0, requests):
                
                # Will Replanish the resources 
                if request != 0 and request % 10 == 0:
                        replenishNetwork(network=network, edges=real_edges, number_of_entanglements=entanglements_replanished)

                # Defining the nodes
                alice, bob = selectAliceBob(network=network, black_hole_list=black_hole_list)

                # Defining route
                route = getRoute(network=network, controller=controller, alice=alice, bob=bob)

                # Create request
                entangled, attempts_counter = createRequest(network=network, alice=alice, bob=bob, attempts=attempts_per_request, route=route)

                # Collect request data
                data['Requests'][f"request:{request}"] = {"Alice & Bob": (alice.host_id, bob.host_id), "Route": route, "Entangled": entangled, "Attempts": attempts_counter}

        # Add eprs data
        data["Used Eprs"] = network.get_total_useds_eprs()
        
        return data

## Rodando a simulação e coletando os dados

In [ ]:
data = simulation(log=False,
    rows=4,
    columns=3,
    entanglements_replanished=1,
    requests=100,
    attempts_per_request=2,
    network_prob=0.8,
    num_black_holes=1,
    black_hole_prob=0.1)

## Mostrando os dados coletados

In [ ]:
data

## Taxa de Sucesso

In [ ]:
success = impossible = fail = 0
for run in data['Requests']:
    if data['Requests'][run]['Entangled'] == 1:
        success += 1
    if data['Requests'][run]['Entangled'] == -1:
        impossible += 1
    if data['Requests'][run]['Entangled'] == 0:
        fail += 1

runs = len(data['Requests'].keys())

success_tax = (success/runs) * 100
impossible_tax = (impossible/runs) * 100
fail_tax = (fail/runs) * 100
print(f'Taxa de sucesso foi de {success_tax}%')
print(f'Taxa de erros foi de {fail_tax}%')
print(f'Taxa de erros por falta de recursos foi de {impossible_tax}%')
print(f"Foram usados no total {data['Used Eprs']} em {len(data['Requests'].keys())} execuções")